<a href="https://colab.research.google.com/github/wandb/edu/blob/main/llm-apps-course/notebooks/02.%20Generation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>
<!--- @wandbcode{llmapps-generation} -->

# Generation
<!--- @wandbcode{llmapps-generation} -->

In this notebook we will dive deeper on prompting the model by passing a better context by using available data from users questions and using the documentation files to generate better answers.


### Setup

In [1]:
# %pip install -Uqqq rich openai==0.27.2 tiktoken wandb tenacity

In [2]:
import os
import random

import openai
import tiktoken

from pathlib import Path
from pprint import pprint
from getpass import getpass

from rich.markdown import Markdown
import pandas as pd
from tenacity import (
    retry,
    stop_after_attempt,
    wait_random_exponential, # for exponential backoff
)  
import wandb
from wandb.integration.openai import autolog

import api_key

In [3]:
# Download files on colab
# if not Path("examples.txt").exists():
#     !wget https://raw.githubusercontent.com/wandb/edu/main/llm-apps-course/notebooks/{examples,prompt_template,system_template}.txt

You will need an OpenAI API key to run this notebook. You can get one [here](https://platform.openai.com/account/api-keys).

In [4]:
# if os.getenv("OPENAI_API_KEY") is None:
#   if any(['VSCODE' in x for x in os.environ.keys()]):
#     print('Please enter password in the VS Code prompt at the top of your VS Code window!')
#   os.environ["OPENAI_API_KEY"] = getpass("Paste your OpenAI key from: https://platform.openai.com/account/api-keys\n")
#   openai.api_key = os.getenv("OPENAI_API_KEY", "")

# assert os.getenv("OPENAI_API_KEY", "").startswith("sk-"), "This doesn't look like a valid OpenAI API key"
# print("OpenAI API key configured")
os.environ["OPENAI_API_KEY"] = api_key.open_ai_key
openai.api_key = os.getenv("OPENAI_API_KEY")

Let's enable W&B autologging to track our experiments.

In [5]:
# start logging to W&B
autolog({"project":"llmapps", "job_type": "generation"})

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: kmirijan. Use `wandb login --relogin` to force relogin


# Generating synthetic support questions

We will add a retry behavior in case we hit the API rate limit

In [6]:
@retry(wait=wait_random_exponential(min=1, max=60), stop=stop_after_attempt(6))
def completion_with_backoff(**kwargs):
    return openai.ChatCompletion.create(**kwargs)

In [30]:
MODEL_NAME = "gpt-3.5-turbo"
# MODEL_NAME = "gpt-4"

In [31]:
system_prompt = "You are a helpful assistant."
user_prompt = "Generate a support question from a W&B user"

def generate_and_print(system_prompt, user_prompt, n=5):
    messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ]
    responses = completion_with_backoff(
        model=MODEL_NAME,
        messages=messages,
        n = n,
        )
    for response in responses.choices:
        generation = response.message.content
        display(Markdown(generation))
    
generate_and_print(system_prompt, user_prompt)

"Can you help me understand how to correctly log food portions in the Weight & Balance app to ensure accurate      
tracking of my daily intake?"

Sure! Here's a support question that a user might ask related to weight and balance calculations for aircraft:     

"How do I account for the weight distribution of passengers and cargo when calculating the center of gravity for my
aircraft?"

"How can I accurately track and manage my daily water intake using the W&B app?"

"Can you provide guidance on how to properly interpret the weight and balance calculations for my aircraft?"

"What is the maximum weight limit for carry-on baggage on your flights?"

# Few Shot 

Let's read some user submitted queries from the file `examples.txt`. This file contains multiline questions separated by tabs (`\t`).

In [32]:
# Test if examples.txt is present, download if not
# if not Path("examples.txt").exists():
#     !wget https://raw.githubusercontent.com/wandb/edu/main/llm-apps-course/notebooks/examples.txt

In [33]:
delimiter = "\t" # tab separated queries
with open("examples.txt", "r") as file:
    data = file.read()
    real_queries = data.split(delimiter)

pprint(f"We have {len(real_queries)} real queries:")  
Markdown(f"Sample one: \n\"{random.choice(real_queries)}\"")

'We have 228 real queries:'


Sample one: "can you tell me how to use W&B sweep together with pytorch's DistributedDataParallel"

We can now use those real user questions to guide our model to produce synthetic questions like those.

In [35]:
def generate_few_shot_prompt(queries, n=3):
    prompt = "Generate a support question from a W&B user\n" +\
        "Below you will find a few examples of real user queries:\n"
    for _ in range(n):
        prompt += random.choice(queries) + "\n"
    prompt += "Let's start!"
    return prompt

generation_prompt = generate_few_shot_prompt(real_queries)
Markdown(generation_prompt)


Generate a support question from a W&B user Below you will find a few examples of real user queries: how do I fix  
an error with wandb Table construction from pandas dataframe: TypeError: Data row contained incompatible types can 
i use wandb with slurm? I'm getting a "Importing a module script failed. An application error occurred." error on a
computer, but the same link works on my phone. Let's start!

OpenAI `Chat` models are really good at following instructions with a few examples. Let's see how it does here. This is going to use some context from the prompt.

In [36]:
generate_and_print(system_prompt, user_prompt=generation_prompt)

Sure! Here's another example of a support question from a W&B user:                                                

"I am trying to use W&B's sweep functionality for hyperparameter optimization, but I keep encountering errors when 
setting up the configurations for my sweep. Can you provide guidance on how to troubleshoot this issue?"

How do I resolve the issue of my wandb Table construction from pandas dataframe resulting in a TypeError: Data row 
contained incompatible types?

How can I resolve the issue of my wandb Table construction from a pandas dataframe resulting in the error:         
TypeError: Data row contained incompatible types?

How can I track multiple metrics from different sources concurrently in WandB?

How do I troubleshoot the "ValueError: Column name collides with a field name: step" issue when trying to log data 
using W&B in my Python script?

# Add Context & Response

Let's create a function to find all the markdown files in a directory and return it's content and path

In [37]:
# check if directory exists, if not, create it and download the files, e.g if running in colab
# if not os.path.exists("../docs_sample/"):
#   !git clone https://github.com/wandb/edu.git
#   !cp -r edu/llm-apps-course/docs_sample ../

In [38]:
def find_md_files(directory):
    "Find all markdown files in a directory and return their content and path"
    md_files = []
    for file in Path(directory).rglob("*.md"):
        with open(file, 'r', encoding='utf-8') as md_file:
            content = md_file.read()
        md_files.append((file.relative_to(directory), content))
    return md_files

documents = find_md_files('../docs_sample/')
len(documents)

11

Let's check if the documents are not too long for our context window. We need to compute the number of tokens in each document.

In [39]:
tokenizer = tiktoken.encoding_for_model(MODEL_NAME)
tokens_per_document = [len(tokenizer.encode(document)) for _, document in documents]
pprint(tokens_per_document)

[803, 1644, 2596, 2093, 365, 1206, 2940, 4179, 2529, 537, 956]


Some of them are too long - instead of using entire documents, we'll extract a random chunk from them

In [40]:
# extract a random chunk from a document
def extract_random_chunk(document, max_tokens=512):
    tokens = tokenizer.encode(document)
    if len(tokens) <= max_tokens:
        return document
    start = random.randint(0, len(tokens) - max_tokens)
    end = start + max_tokens
    return tokenizer.decode(tokens[start:end])

Now, we will use that extracted chunk to create a question that can be answered by the document. This way we can generate questions that our current documentation is capable of answering.

In [41]:
def generate_context_prompt(chunk):
    prompt = "Generate a support question from a W&B user\n" +\
        "The question should be answerable by provided fragment of W&B documentation.\n" +\
        "Below you will find a fragment of W&B documentation:\n" +\
        chunk + "\n" +\
        "Let's start!"
    return prompt

chunk = extract_random_chunk(documents[0][1])
generation_prompt = generate_context_prompt(chunk)

In [42]:
Markdown(generation_prompt)

Generate a support question from a W&B user The question should be answerable by provided fragment of W&B          
documentation. Below you will find a fragment of W&B documentation: to store whatever files are produced from the  
serialization process provided by your modeling library (eg ]8;id=207392;https://pytorch.org/tutorials/beginner/saving_loading_models.html\PyTorch]8;;\ & ]8;id=799415;https://www.tensorflow.org/guide/keras/save_and_serialize\Keras]8;;\).                                      

Furthermore, a Model Artifact is a sequence of Model Versions. Model Artifact can alias specific versions so that  
downstream consumers can pin to such aliases. It is extremely common for a W&B Run to produce many versions of a   
model while training (periodically saving checkpoints). Using this approach, each individual model being trained by
the Run corresponds to its own Model Artifact, and each checkpoint corresponds to its own Model Version of the     
respective Model Artifact.                                                                                         

View an ]8;id=492290;https://wandb.ai/timssweeney/model_management_docs_official_v0/artifacts/model/mnist-zws7gt0n\example Model Artifact ->]8;;\                                                                                  

🌆 ]8;id=197299;@site/static/images/models/mr1c.png\mr1c.png]8;;\                                                                                                                    

Finally, a Registered Model is a set of links to Model Versions. A Registered Model can be accessed exactly like   
Model Artifacts (identified by [[entityName/]/projectName]/registeredModelName:alias), however it acts more like a 
folder of "bookmarks" - where each "version" of a Registered Model is actually a link to an Model Version belonging
to a Model Artifact of the same type. A Model Version may be linked to any number of Registered Models. Typically  
you will create a Registered Model for each of your use cases / modeling tasks and use aliases like "production" or
"staging" to denote versions with special purposes.                                                                

View an ]8;id=635987;https://wandb.ai/timssweeney/model_management_docs_official_v0/artifacts/model/MNIST%20Grayscale%2028x28\example Registered Model ->]8;;\                                                                                

🌆 ]8;id=704886;/images/models/diagram_doc.png\diagram_doc.png]8;;\                                                                                                                    

While developing an ML Model, you will likely have dozens, hundreds, or even thousands of Runs which produce Model 
Versions - they may come from notebooks, remote training jobs, CI/CD pipelines, etc... Most likely, not all of     
those models are great; often you are iterating on scripts, parameters, architectures, preprocessing logic and     
more. The separation of Artifacts and Registered Models allows you to produce a massive number of Artifacts (think 
of them like "draft models"), and periodically link your high performing versions to a the curated Registered      
Model. Then use Let's start!

Let's generate 3 possible questions:

In [43]:
generate_and_print(system_prompt, generation_prompt, n=3)

Certainly! A support question related to the provided fragment of W&B documentation could be:                      

"How can I link my high-performing model versions to a curated Registered Model in Weights & Biases?"              

This question can be answered by referring to the concept described in the documentation where a Registered Model  
is defined as a set of links to Model Versions. It explains that you can create a Registered Model for each of your
use cases or modeling tasks and use aliases like "production" or "staging" to denote versions with specific        
purposes. Each "version" of a Registered Model is actually a link to a Model Version belonging to a Model Artifact 
of the same type.

User question: How can I store and manage different versions of models produced during training in W&B?            

Answer: You can store different versions of models produced during training in W&B using Model Artifacts and       
Registered Models. A Model Artifact is a sequence of Model Versions, where each checkpoint corresponds to its own  
Model Version of the respective Model Artifact. On the other hand, a Registered Model is a set of links to Model   
Versions, acting like a folder of bookmarks. You can create a Registered Model for each of your use cases or       
modeling tasks and use aliases like "production" or "staging" to denote versions with special purposes. This allows
you to produce a massive number of Artifacts (like "draft models") and periodically link your high-performing      
versions to a curated Registered Model.

User support question: How can I create and manage Registered Models in W&B for my machine learning projects?      

Answer: You can create and manage Registered Models in W&B by using aliases to denote versions with special        
purposes, such as "production" or "staging." Registered Models are sets of links to Model Versions, acting like    
folders of bookmarks that are linked to Model Versions belonging to Model Artifacts. Each Model Version can be     
linked to any number of Registered Models, allowing you to organize and curate your models effectively.

> As you can see, sometimes the generation contains an intro phrase like: "Sure, here's a support question based on the documentation:", we may want to put some instructions to avoid this.

### Level 5 prompt

Complex directive that includes the following:
- Description of high-level goal
- A detailed bulleted list of sub-tasks
- An explicit statement asking LLM to explain its own output
- A guideline on how LLM output will be evaluated
- Few-shot examples

In [44]:
# we will use GPT4 from here, as it gives better answers and abides to instructions better
MODEL_NAME = "gpt-4"
# MODEL_NAME = "gpt-3.5-turbo"

In [45]:
# read system_template.txt file into an f-string
with open("system_template.txt", "r") as file:
    system_prompt = file.read()

In [46]:
Markdown(system_prompt)

You are a creative assistant with the goal to generate a synthetic dataset of Weights & Biases (W&B) user          
questions. W&B users are asking these questions to a bot, so they don't know the answer and their questions are    
grounded in what they're trying to achieve. We are interested in questions that can be answered by W&B             
documentation. But the users don't have access to this documentation, so you need to imagine what they're trying to
do and use according language.

In [47]:
# read prompt_template.txt file into an f-string
with open("prompt_template.txt", "r") as file:
    prompt_template = file.read()

In [48]:
Markdown(prompt_template)

Here are some examples of real user questions, you will be judged by how well you match this distribution.         

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{QUESTIONS}                                                                                                        

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
In the next step, you will read a fragment of W&B documentation. This will serve as inspiration for synthetic user 
question and the source of the answer. Here is the document fragment:                                              

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
{CHUNK}                                                                                                            

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
You will now generate a user question and corresponding answer based on the above document. First, explain the user
context and what problems they might be trying to solve. Second, generate user question. Third, provide the        
accurate and concise answer in markdown format to the user question using the documentation. You'll be evaluated   
on:                                                                                                                

 • how realistic is that this question will come from a real user one day?                                         
 • is this question about W&B?                                                                                     
 • can the question be answered using the W&B document fragment above?                                             
 • how accurate is the answer? Remember that users have different styles and can be imprecise. You are very good at
   impersonating them! Use the following format: CONTEXT: QUESTION: ANSWER: Let's start!

In [49]:
def generate_context_prompt(chunk, n_questions=3):
    questions = '\n'.join(random.sample(real_queries, n_questions))
    user_prompt = prompt_template.format(QUESTIONS=questions, CHUNK=chunk)
    return user_prompt

user_prompt = generate_context_prompt(chunk)

In [50]:
Markdown(user_prompt)

Here are some examples of real user questions, you will be judged by how well you match this distribution.         

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
The movement is choppy when I drag either horizontally or vertically the wandb table. It seems that it takes a long
time loading blocks of data. Is there a way to accomodate this? How to export a single chart's data using the API? 
@wandbot (beta) While running a sweep, runs fail with FileNotFoundError(2, 'No such file or directory'). I took a  
look at the logs for those exact runs, but there is no error to be found there. In addition, once that error       
happened for a agent, it keeps happening over and over, even if the agent before had successful runs. How do I     
debug this?                                                                                                        

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
In the next step, you will read a fragment of W&B documentation. This will serve as inspiration for synthetic user 
question and the source of the answer. Here is the document fragment:                                              

───────────────────────────────────────────────────────────────────────────────────────────────────────────────────
to store whatever files are produced from the serialization process provided by your modeling library (eg ]8;id=220838;https://pytorch.org/tutorials/beginner/saving_loading_models.html\PyTorch]8;;\ &
]8;id=421084;https://www.tensorflow.org/guide/keras/save_and_serialize\Keras]8;;\).                                                                                                            

Furthermore, a Model Artifact is a sequence of Model Versions. Model Artifact can alias specific versions so that  
downstream consumers can pin to such aliases. It is extremely common for a W&B Run to produce many versions of a   
model while training (periodically saving checkpoints). Using this approach, each individual model being trained by
the Run corresponds to its own Model Artifact, and each checkpoint corresponds to its own Model Version of the     
respective Model Artifact.                                                                                         

View an ]8;id=98737;https://wandb.ai/timssweeney/model_management_docs_official_v0/artifacts/model/mnist-zws7gt0n\example Model Artifact ->]8;;\                                                                                  

🌆 ]8;id=714704;@site/static/images/models/mr1c.png\mr1c.png]8;;\                                                                                                                    

Finally, a Registered Model is a set of links to Model Versions. A Registered Model can be accessed exactly like   
Model Artifacts (identified by [[entityName/]/projectName]/registeredModelName:alias), however it acts more like a 
folder of "bookmarks" - where each "version" of a Registered Model is actually a link to an Model Version belonging
to a Model Artifact of the same type. A Model Version may be linked to any number of Registered Models. Typically  
you will create a Registered Model for each of your use cases / modeling tasks and use aliases like "production" or
"staging" to denote versions with special purposes.                                                                

View an ]8;id=639271;https://wandb.ai/timssweeney/model_management_docs_official_v0/artifacts/model/MNIST%20Grayscale%2028x28\example Registered Model ->]8;;\                                                                                

🌆 ]8;id=812949;/images/models/diagram_doc.png\diagram_doc.png]8;;\                                                                                                                    

While developing an ML Model, you will likely have dozens, hundreds, or even

In [51]:
def generate_questions(documents, n_questions=3, n_generations=5):
    questions = []
    for _, document in documents:
        chunk = extract_random_chunk(document)
        user_prompt = generate_context_prompt(chunk, n_questions)
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ]
        response = completion_with_backoff(
            model=MODEL_NAME,
            messages=messages,
            n = n_generations,
            )
        questions.extend([response.choices[i].message.content for i in range(n_generations)])
    return questions

> A Note about the `system` role: For GPT4 based pipelines you probably want to move some part of the context prompt to the `system` context. As we are using `gpt3.5-turbo` here, you can put the instruction on the user prompt, you can read more about this on [OpenAI docs here](https://platform.openai.com/docs/guides/chat/instructing-chat-models)

In [52]:
# function to parse model generation and extract CONTEXT, QUESTION and ANSWER
def parse_generation(generation):
    lines = generation.split("\n")
    context = []
    question = []
    answer = []
    flag = None
    
    for line in lines:
        if "CONTEXT:" in line:
            flag = "context"
            line = line.replace("CONTEXT:", "").strip()
        elif "QUESTION:" in line:
            flag = "question"
            line = line.replace("QUESTION:", "").strip()
        elif "ANSWER:" in line:
            flag = "answer"
            line = line.replace("ANSWER:", "").strip()

        if flag == "context":
            context.append(line)
        elif flag == "question":
            question.append(line)
        elif flag == "answer":
            answer.append(line)

    context = "\n".join(context)
    question = "\n".join(question)
    answer = "\n".join(answer)
    return context, question, answer

In [53]:
generations = generate_questions([documents[0]], n_questions=3, n_generations=5)
parse_generation(generations[0])

('\nA user has just started using Weights & Biases. They have hundreds of runs that have resulted in different versions of their ML models. They want to understand more about managing model versions and how can they categorize all the different model versions (each suited for a unique use case) in W&B. \n',
 '\nI have a bunch of ML model versions bottled up from numerous runs. How can I manage and categorize these in different sets, each tagged for a particular use case, in Weights & Biases?\n',
 '\nIn Weights & Biases, you can manage and categorize your model versions using **Model Artifacts** and **Registered Models**.\n\nA **Model Artifact** is a sequence of Model Versions produced from different training runs. Each individual model being trained corresponds to its own Model Artifact, and each checkpoint during the training corresponds to a Model Version of that respective Model Artifact. You can also **alias** specific versions so that you can easily pin and track them. \n\nA **Reg

In [54]:
parsed_generations = []
generations = generate_questions(documents, n_questions=3, n_generations=5)
for generation in generations:
    context, question, answer = parse_generation(generation)
    parsed_generations.append({"context": context, "question": question, "answer": answer})

# let's convert parsed_generations to a pandas dataframe and save it locally
df = pd.DataFrame(parsed_generations)
df.to_csv('generated_examples.csv', index=False)

# log df as a table to W&B for interactive exploration
wandb.log({"generated_examples": wandb.Table(dataframe=df)})

# log csv file as an artifact to W&B for later use
artifact = wandb.Artifact("generated_examples", type="dataset")
artifact.add_file("generated_examples.csv")
wandb.log_artifact(artifact)

<Artifact generated_examples>

In [55]:
wandb.finish()